In [ ]:
#Installation
#Install the following packages for executing this notebook.

In [5]:
# install packages
#! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                    #google-cloud-storage \
                                    #jsonlines 

In [42]:
pip install google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


In [3]:
#! pip3 install --force-reinstall 'google-cloud-aiplatform>=1.15' -q --no-warn-conflicts


In [10]:
#Set project ID

In [43]:

PROJECT_ID = "training1emakia"  # @param {type:"string"}

In [44]:
print(PROJECT_ID)

training1emakia


In [8]:
#! gcloud auth login

In [45]:

# set the project id
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [46]:
# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [47]:
! gcloud config list

[core]
account = 12807912884-compute@developer.gserviceaccount.com
disable_usage_reporting = True
project = training1emakia

Your active configuration is: [default]


In [10]:
#Set Region

In [31]:
REGION = "us-central1"  # @param {type: "string"}

In [ ]:
#Create a Cloud Storage bucket
#Create a storage bucket to store intermediate artifacts such as datasets.

In [48]:
BUCKET_NAME = f"{MY_NAME}tweetsdatatraining1"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [49]:
! gsutil mb -l us-central1 gs://{BUCKET_NAME}

Creating gs://{MY_NAME}tweetsdatatraining1/...
ServiceException: 409 A Cloud Storage bucket named '{MY_NAME}tweetsdatatraining1' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [50]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'12807912884-compute@developer.gserviceaccount.com'

In [51]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

No changes made to gs://{MY_NAME}tweetsdatatraining1/
No changes made to gs://{MY_NAME}tweetsdatatraining1/


In [ ]:
#Initialize Vertex AI
#Initialize the Vertex AI SDK for Python for the  project.

In [52]:
from google.cloud import aiplatform

In [53]:

from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [54]:
display_name = f"{MY_NAME}2-english-tweetsdata-{TIMESTAMP}"

In [55]:
print(display_name)

{MY_NAME}2-english-tweetsdata-20231122022517


In [40]:
#Create a Dataset resource and import the data
#Using the Python SDK, we create a dataset and import the dataset in one call to TextDataset.create(), 
#as shown in the following cell.

#Creating and importing data is a long-running operation. 

In [56]:
# Use a timestamp to ensure unique resources
src_uris = "gs://data_{MY_NAME}/tweets-labels.csv"

text_dataset = aiplatform.TextDataset.create(
    display_name=display_name,
    gcs_source=src_uris,
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.single_label_classification,
    sync=True,
)
text_dataset.wait()
    
print(text_dataset.display_name)
print(text_dataset.resource_name)

Creating TextDataset
Create TextDataset backing LRO: projects/12807912884/locations/us-central1/datasets/5913324167272333312/operations/8501117623256219648
TextDataset created. Resource name: projects/12807912884/locations/us-central1/datasets/5913324167272333312
To use this TextDataset in another session:
ds = aiplatform.TextDataset('projects/12807912884/locations/us-central1/datasets/5913324167272333312')
Importing TextDataset data: projects/12807912884/locations/us-central1/datasets/5913324167272333312
Import TextDataset data backing LRO: projects/12807912884/locations/us-central1/datasets/5913324167272333312/operations/3249920457742221312
TextDataset data imported. Resource name: projects/12807912884/locations/us-central1/datasets/5913324167272333312
{MY_NAME}2-english-tweetsdata-20231122022517
projects/12807912884/locations/us-central1/datasets/5913324167272333312


In [ ]:
#Train the text classification model
#Training the model is a two part process:

#Define the training job. You must provide a display name and the type of training you want when you define the training job.
#Run the training job. 
#When running the training job, you need to supply a reference to the dataset to use for training. 
#At this step, you can also configure the data split percentages.
#You do not need to specify data splits. 
#The training job has a default setting of training 80%/ testing 10%/ validate 10% if you don't provide these values.

#To train your model, you call AutoMLTextTrainingJob.run().
#The method returns a reference to the new Model object.

#As with importing data into the dataset, training your model can take a substantial amount of time. 
#The client library prints out operation status messages while the training pipeline operation processes. 
#We must wait for the training process to complete before we can get the resource name and ID of the new model, 
#which is required for model evaluation and model deployment.


In [57]:
# Define the training job
training_job_display_name = f"{MY_NAME}2-english-tweet-text-training-job-{TIMESTAMP}"
job = aiplatform.AutoMLTextTrainingJob(
    display_name=training_job_display_name,
    prediction_type="classification",
    multi_label=False,
)

In [71]:
text_dataset


resource name: projects/12807912884/locations/us-central1/datasets/5913324167272333312

In [59]:
text_dataset = aiplatform.TextDataset("5913324167272333312")

In [69]:
text_dataset

resource name: projects/12807912884/locations/us-central1/datasets/5913324167272333312

In [70]:
# Run the Training job
model_display_name = f"{MY_NAME}2-tweet-text-classification-model-{TIMESTAMP}"

#text_dataset = aiplatform.TextDataset("9181496036999299072")

model = job.run(
    dataset= text_dataset,
    model_display_name=model_display_name,
    training_fraction_split=0.7,
    validation_fraction_split=0.2,
    test_fraction_split=0.1,
    sync=True
)

RuntimeError: AutoML Text Training has already run.

In [62]:
model_display_name

'{MY_NAME}2-tweet-text-classification-model-20231122022517'

In [63]:
model = aiplatform.Model.list(filter=f'display_name={model_display_name}')

In [64]:
model[0]

resource name: projects/12807912884/locations/us-central1/models/7904114214174720000

In [ ]:
# Review model evaluation scores
# After the model training has finished, we can review the evaluation scores for it using the list_model_evaluations() method. 
# This method will return an iterator for each evaluation slice.

In [65]:
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
deployed_model_display_name = f"{MY_NAME}-deployed-tweets-text-classification-model-{timestamp}"
endpoint = model[0].deploy(
    deployed_model_display_name=deployed_model_display_name, sync=True
)

Creating Endpoint
Create Endpoint backing LRO: projects/12807912884/locations/us-central1/endpoints/8493069816616189952/operations/6158893973302673408
Endpoint created. Resource name: projects/12807912884/locations/us-central1/endpoints/8493069816616189952
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/12807912884/locations/us-central1/endpoints/8493069816616189952')
Deploying model to Endpoint : projects/12807912884/locations/us-central1/endpoints/8493069816616189952
Deploy Endpoint model backing LRO: projects/12807912884/locations/us-central1/endpoints/8493069816616189952/operations/3853050964088979456
Endpoint model deployed. Resource name: projects/12807912884/locations/us-central1/endpoints/8493069816616189952


In [68]:
model_evaluations = model[0].list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

{'name': 'projects/12807912884/locations/us-central1/models/7904114214174720000@1/evaluations/2911000980642332672', 'metricsSchemaUri': 'gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml', 'metrics': {'confusionMatrix': {'rows': [[2244.0, 284.0], [166.0, 4003.0]], 'annotationSpecs': [{'displayName': '0', 'id': '1000665807316844544'}, {'displayName': '1', 'id': '5612351825744232448'}]}, 'confidenceMetrics': [{'precisionAt1': 0.9328057, 'precision': 0.5, 'recallAt1': 0.9328057, 'f1ScoreAt1': 0.9328057, 'f1Score': 0.6666667, 'recall': 1.0, 'maxPredictions': 0.0, 'confidenceThreshold': 0.0}, {'precision': 0.77400535, 'maxPredictions': 0.0, 'f1ScoreAt1': 0.9328057, 'confidenceThreshold': 0.05, 'recallAt1': 0.9328057, 'f1Score': 0.86900705, 'recall': 0.9905928, 'precisionAt1': 0.9328057}, {'precisionAt1': 0.9328057, 'confidenceThreshold': 0.1, 'recallAt1': 0.9328057, 'maxPredictions': 0.0, 'precision': 0.82359564, 'f1ScoreAt1': 0.9328057, 'f1Score': 0.8962